In [1]:
import sqlite3
import pandas as pd

In [2]:
cnn = sqlite3.connect("library.db")
c = cnn.cursor()

### Challenge 1

Determine how many copies of the book 'Dracula' are available for library patrons to borrow.

In [3]:
query = """
        SELECT COUNT(*)
        FROM Books
        WHERE Title = 'Dracula'
"""
pd.read_sql_query(query, cnn)

# there are 3 copies of Dracula

,COUNT(*)
0,3


In [4]:
query = """
        SELECT COUNT(Books.Title)
        FROM Loans
        JOIN Books ON Loans.BookID = Books.BookID
        WHERE Books.Title = 'Dracula' AND
              Loans.ReturnedDate IS NULL
"""
pd.read_sql_query(query, cnn)

# how many copies haven't been returned

,COUNT(Books.Title)
0,1


In [5]:
# combining these two statements
query = """
SELECT
        (SELECT COUNT(Books.Title)
        FROM Books
        WHERE Books.Title = 'Dracula') -
        (SELECT COUNT(Books.Title)
        FROM Loans
        JOIN Books ON Loans.BookID = Books.BookID
        WHERE Books.Title = 'Dracula' AND
              Loans.ReturnedDate IS NULL)
        AS AvailableCopies
"""
pd.read_sql_query(query, cnn)

,AvailableCopies
0,2


### Challenge 2

Add to the Books table

In [6]:
query = """
        INSERT INTO Books(Title, Author, Published, Barcode)
        VALUES
        ("Dracula", "Bram Stoker", 1897, 4819277482),
        ("Gulliver's Travels", "Jonathan Swift", 1729, 4899254401)
"""
c.execute(query)

In [7]:
query = """
        SELECT *
        FROM Books
        ORDER BY BookID DESC
        LIMIT 5
"""
pd.read_sql_query(query, cnn)

,BookID,Title,Author,Published,Barcode
0,202,Gulliver's Travels,Jonathan Swift,1729,4899254401
1,201,Dracula,Bram Stoker,1897,4819277482
2,200,A Doll's House,Henrik Ibsen,1879,2719805876
3,199,Anna Karenina,graf Leo Tolstoy,1878,6318181860
4,198,A Doll's House,Henrik Ibsen,1879,6227932647


### Challenge 3

Check out these books to their customer

In [8]:
# Book id of The picture of Dorian Gray 
# with 2855934983 barcode
query = """
        SELECT BookID
        FROM Books
        WHERE Barcode = 2855934983
"""
pd.read_sql_query(query, cnn)

,BookID
0,11


In [9]:
# Book id of Great Expectations with 4043822646 barcode
query = """
        SELECT BookID
        FROM Books
        WHERE Barcode = 4043822646
"""
pd.read_sql_query(query, cnn)

,BookID
0,93


In [10]:
query = """
        SELECT PatronID
        FROM Patrons
        WHERE Email = 'jvaan@wisdompets.com'
"""
pd.read_sql_query(query, cnn)

,PatronID
0,50


In [11]:
query = """
        INSERT INTO Loans(BookID, PatronID, LoanDate, DueDate)
        VALUES
        ((SELECT BookID 
            FROM Books 
            WHERE Barcode = 2855934983), 
        (SELECT PatronID
            FROM Patrons
            WHERE Email = 'jvaan@wisdompets.com'), 
        '2022-08-25', 
        '2022-09-08'),
        ((SELECT BookID 
            FROM Books
            WHERE Barcode = 4043822646), 
        (SELECT PatronID
            FROM Patrons 
            WHERE Email = 'jvaan@wisdompets.com'), 
            '2022-08-25',
            '2022-09-08')
        
"""
c.execute(query)

In [12]:
query = """
        SELECT *
        FROM Loans
        ORDER BY LoanID DESC
        LIMIT 5
"""
pd.read_sql_query(query, cnn)

,LoanID,BookID,PatronID,LoanDate,DueDate,ReturnedDate
0,2002,93,50,2022-08-25,2022-09-08,None
1,2001,11,50,2022-08-25,2022-09-08,None
2,2000,91,9,2022-06-30,2022-07-14,None
3,1999,73,59,2022-06-29,2022-07-13,None
4,1998,169,18,2022-06-29,2022-07-13,None


### Challenge 4

Prepare a report of books due to be returned to the library on July 13, 2022. Provide the due date, the book title and the borrower's first name and email address.

In [13]:
query = """
    SELECT b.Title, b.Author, 
           p.FirstName, p.LastName, p.Email,
           l.DueDate, l.ReturnedDate
    FROM Loans as l
    JOIN Books AS b ON b.BookID = l.BookID
    JOIN Patrons AS p ON p.PatronID = l.PatronID
    WHERE l.DueDate = '2022-07-13'
    AND l.ReturnedDate IS NULL
"""
pd.read_sql_query(query, cnn)

,Title,Author,FirstName,LastName,Email,DueDate,ReturnedDate
0,The Count of Monte Cristo,Alexandre Dumas,Kala,Simonian,ksimonian2q@landonhotel.com,2022-07-13,None
1,The Prophet,Kahlil Gibran,Lydie,Awmack,lawmack1a@samoca.org,2022-07-13,None
2,Second Treatise of Government,John Locke,Anthe,Dinjes,adinjesh@samoca.org,2022-07-13,None
3,Dracula,Bram Stoker,Cornelia,Koppe,ckoppe1m@landonhotel.com,2022-07-13,None


###  Challenge 5

Return three books to the library using their bar codes:

- Return date: July 5, 2022
- Book 1: 6435968624
- Book 2: 5677520613
- Book 3: 8730298424

In [14]:
# we need the book IDs
query = """
    SELECT BookID
    FROM Books
    WHERE Barcode in (6435968624, 5677520613, 8730298424)
"""
pd.read_sql_query(query, cnn)

,BookID
0,46
1,73
2,105


In [15]:
# we need the loan id
query = """
        SELECT LoanID
        FROM Loans
        WHERE BookID in (46,73,105)
        AND ReturnedDate IS NULL
"""
pd.read_sql_query(query, cnn)

,LoanID
0,1991
1,1992
2,1999


In [16]:
query = """

    UPDATE Loans
    SET ReturnedDate = '2022-07-13'
    WHERE LoanID in (1992, 1999, 1991)
"""
c.execute(query)

In [17]:
query = """
        SELECT *
        FROM Loans
        WHERE LoanID in (1992, 1999, 1991)
"""
pd.read_sql_query(query, cnn)

,LoanID,BookID,PatronID,LoanDate,DueDate,ReturnedDate
0,1991,105,65,2022-06-25,2022-07-09,2022-07-13
1,1992,46,59,2022-06-25,2022-07-09,2022-07-13
2,1999,73,59,2022-06-29,2022-07-13,2022-07-13


### Challenge 6

Prepate a report of the library patrons who have checked out the fewest books

In [18]:
query = """
        SELECT l.PatronID, COUNT(l.PatronID) AS CountBooks,
               p.FirstName, p.Email
        FROM Loans AS l
        JOIN Patrons AS p ON p.PatronID = l.PatronID
        GROUP BY l.PatronID
        ORDER BY CountBooks
        LIMIT 5
"""
pd.read_sql_query(query, cnn)

,PatronID,CountBooks,FirstName,Email
0,69,9,Alaster,aruggles1w@red30design.com
1,83,9,Caril,cmatejic2a@orangevalleycaa.org
2,73,11,Merle,msukbhans20@orangevalleycaa.org
3,64,12,Norby,nsleight1r@kinetecoinc.com
4,94,12,Curr,cfittall2l@orangevalleycaa.org


### Challenge 7

Create a list of books to feature in an exhibition. Make a pick list of books published from 1890-1899 which are not currently checlked out

In [19]:
query = """
        SELECT Title, Barcode
        FROM Books
        WHERE Published BETWEEN 1890 AND 1899 AND
                BookID NOT IN (SELECT BookID
                                    FROM Loans 
                                    WHERE ReturnedDate IS NULL)
       ORDER BY Title
"""
pd.read_sql_query(query, cnn)

,Title,Barcode
0,Also sprach Zarathustra,8233453745
1,Also sprach Zarathustra,8880867275
2,An Occurrence at Owl Creek Bridge,6432246701
3,An Occurrence at Owl Creek Bridge,6854696214
4,Dracula,8232533734
5,Dracula,4274716715
6,Dracula,8730298424
7,Dracula,4819277482
8,Heart of Darkness,9334362486
9,Heart of Darkness,2229312049


### Challenge 8

Create two reports about book statistics.
- Report 1: Show how many titles were published in each year
- Report 2: Show the five books that have been checked out the most

In [20]:
# Report 1
query = """
        SELECT Published, COUNT(DISTINCT(Title)) AS PubCount, Title
        FROM Books
        GROUP BY Published
        ORDER BY PubCount DESC
        LIMIT 5
"""
pd.read_sql_query(query, cnn)

,Published,PubCount,Title
0,1895,3,The Importance of Being Earnest: A Trivial Com...
1,1890,3,The Sign of the Four
2,1922,2,Ulysses
3,1911,2,Peter Pan
4,1899,2,"The Awakening, and Selected Short Stories"


In [21]:
# Report 2
query = """
        SELECT COUNT(l.LoanID) AS LoanCount, b.Title
        FROM Loans AS l
        JOIN Books AS b ON l.BookID = b.BookID
        GROUP BY b.Title
        ORDER BY LoanCount DESC
        LIMIT 5
"""
pd.read_sql_query(query, cnn)

,LoanCount,Title
0,53,"Frankenstein; Or, The Modern Prometheus"
1,43,The War of the Worlds
2,43,"The Awakening, and Selected Short Stories"
3,42,The Scarlet Letter
4,40,The Picture of Dorian Gray


Clone connections

In [22]:
c.close()
cnn.close()